<a href="https://colab.research.google.com/github/jakkaphat1/Project1_doc/blob/main/(EmbeddingModel%2Bn_gram%2Btemplate%2BCLIPs%2Bmatching_count)_Assignment_Plagiarism_Detection_V3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Document**
Text Embedding Model : LaBse

Lexical Matching Process : N-gram + Jaccard Similarity

Template Removing

------------------------
Image Embedding Model : clip-vit-large-patch14

Embedding similarity choosen : Matching Count (ใช้จำนวนภาพที่มี Similarity > threshold ที่ตั้งไว้)

------------------------
Process bitmap & vector graphics



# Requirements

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

import os
from slugify import slugify

import shutil


In [2]:
!pip install -q pythainlp
from pythainlp.tokenize import word_tokenize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 41.8 MB/s eta 0:00:00


In [3]:
!pip install pdf2image
from pdf2image import convert_from_path


In [4]:
!pip install pymupdf
!pip install pytesseract
!pip install pillow pdf2image
!apt-get install poppler-utils
!apt-get install tesseract-ocr
!apt-get install tesseract-ocr-tha

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 57.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.9 [186 kB]
Fetched 186 kB in 1s (134 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126371 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.9_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.9) ...
Setting up poppler-utils (22.02.0-2ubuntu0.9) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr

In [5]:
!pip install pinecone-client transformers pymupdf


In [6]:
pip install python-slugify

In [7]:
pip install sentence-transformers


In [8]:
pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [9]:
import fitz  # PyMuPDF
import pytesseract
from pdf2image import convert_from_path
from PIL import Image

pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

In [10]:
from google.colab import files

In [11]:
import tempfile

# Embedding Models

In [12]:
import fitz  # PyMuPDF
import tempfile
import os
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
from sentence_transformers import SentenceTransformer

# โหลดโมเดล Embedding
labse_model = SentenceTransformer("sentence-transformers/LaBSE")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

# Pinecone

In [13]:
from pinecone import Pinecone, ServerlessSpec
# Pinecone API Key และ Environment (Region)
PINECONE_API_KEY = "pcsk_3q7EHL_2jLmXAfQ9rXqj3h2tHaG1YjjYuQ5HnEtNHucWasCCsxdvr8qvLHqPip4Aa17ziJ"
PINECONE_ENVIRONMENT = "us-east-1"  # Region

# สร้าง Pinecone Client Instance
pc = Pinecone(api_key=PINECONE_API_KEY)

# ตรวจสอบ Index ก่อนสร้าง
if "text-index" in pc.list_indexes().names():
    print(pc.list_indexes().names())

if "image-index" in pc.list_indexes().names():
    print(pc.list_indexes().names())

# โหลด Index
text_index = pc.Index("text-index")
image_index = pc.Index("image-index")



['text-1024', 'text-index', 'text-769', 'image-1024', 'image-index']
['text-1024', 'text-index', 'text-769', 'image-1024', 'image-index']


# Plagiarism Process

In [143]:
from google.colab import files

def upload_pdfs():
    uploaded = files.upload()
    pdf_paths = list(uploaded.keys())
    return pdf_paths

pdf_paths = upload_pdfs()

Saving 663380129-5.pdf to 663380129-5.pdf
Saving 663380446-4.pdf to 663380446-4.pdf
Saving 663380446-5.pdf to 663380446-5.pdf


In [144]:
def extract_text_and_embed(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc]).strip()

    if not text:
        # ใช้ OCR กรณีที่ PyMuPDF ไม่เห็นข้อความ
        images = convert_from_path(pdf_path)
        ocr_text = []
        for img in images:
            ocr_text.append(pytesseract.image_to_string(img, lang="tha+eng"))
        text = "\n".join(ocr_text).strip()

    #ตัดคำด้วย PyThaiNLP
    tokens = word_tokenize(text, engine="newmm")
    tokenized_text = " ".join(tokens)


    text_embedding = labse_model.encode(tokenized_text)

    return tokenized_text, text_embedding

###################################
def extract_images_and_embed(pdf_path):
    doc = fitz.open(pdf_path)
    image_embeddings = []
    temp_dir = tempfile.mkdtemp()

    for page_num in range(len(doc)):
        # --- STEP 1: Try embedded images first ---
        embedded_found = False
        for img_index, img_info in enumerate(doc[page_num].get_images(full=True)):
            xref = img_info[0]
            base_image = doc.extract_image(xref)
            img_bytes = base_image["image"]
            img_path = os.path.join(temp_dir, f"page_{page_num}_img_{img_index}.png")
            with open(img_path, "wb") as f:
                f.write(img_bytes)
            img = Image.open(img_path).convert("RGB")
            inputs = clip_processor(images=img, return_tensors="pt")
            with torch.no_grad():
                img_embedding = clip_model.get_image_features(**inputs)
            image_embeddings.append(img_embedding.squeeze().numpy())
            embedded_found = True

        # --- STEP 2: Fallback: Render page as high-res bitmap if no embedded image ---
        if not embedded_found:
            mat = fitz.Matrix(3.0, 3.0)  # zoom 3x เพื่อ high-res
            pix = doc[page_num].get_pixmap(matrix=mat)
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

            inputs = clip_processor(images=img, return_tensors="pt")
            with torch.no_grad():
                img_embedding = clip_model.get_image_features(**inputs)
            image_embeddings.append(img_embedding.squeeze().numpy())

    shutil.rmtree(temp_dir)
    return image_embeddings


In [145]:
from difflib import SequenceMatcher

def remove_template_text(student_text, template_text, threshold=0.9, return_removed=False):
    student_lines = student_text.strip().split('\n')
    template_lines = template_text.strip().split('\n')

    result = []
    removed = []

    for s_line in student_lines:
        match_found = any(
            SequenceMatcher(None, s_line.strip(), t_line.strip()).ratio() > threshold
            for t_line in template_lines
        )
        if not match_found:
            result.append(s_line)
        else:
            removed.append(s_line)

    if return_removed:
        return "\n".join(result).strip(), "\n".join(removed).strip()
    else:
        return "\n".join(result).strip()


In [146]:
def to_ascii_id(filename):
    return slugify(filename, separator="_")

In [148]:
def upload_text_embedding(doc_id, text_embedding):
    if text_embedding is not None and len(text_embedding) > 0:
        text_index.upsert([(f"text_{doc_id}", text_embedding.tolist(), {"type": "text"})])

def upload_image_embedding(doc_id, image_embeddings):
    if image_embeddings:
        for i, img_emb in enumerate(image_embeddings):
            image_index.upsert([(f"image_{doc_id}_{i}", img_emb.tolist(), {"type": "image"})])

In [149]:
def matching_count(image_embeddings, doc_id_1, doc_id_2, threshold=0.8):
    similarities = []
    keys1 = [k for k in image_embeddings if k.startswith(f"image_{doc_id_1}_")]
    keys2 = [k for k in image_embeddings if k.startswith(f"image_{doc_id_2}_")]

    for k1 in keys1:
        for k2 in keys2:
            sim = cosine_similarity([image_embeddings[k1]], [image_embeddings[k2]])[0][0]
            similarities.append(sim)

    # นับจำนวนคู่ที่ similarity มากกว่า threshold
    matched = [sim for sim in similarities if sim >= threshold]

    # คำนวณ Matching Ratio: จำนวนคู่ที่ตรงเกณฑ์ / จำนวนคู่ทั้งหมด
    matching_ratio = len(matched) / len(similarities) if similarities else 0.0

    return matching_ratio


In [150]:
# ===== เลือกโหมดการใช้ Template หลังอัปโหลดไฟล์นักศึกษา =====
print("ต้องการใช้ไฟล์ Template ของอาจารย์เพื่อลบข้อความซ้ำหรือไม่?")
print("0 = ไม่ใช้ template")
print("1 = ใช้ template")
try:
    USE_TEMPLATE = int(input("เลือกโหมด 0 หรือ 1 : "))
except:
    USE_TEMPLATE = 0

template_text_raw = ""
if USE_TEMPLATE == 1:
    print("โปรดอัปโหลดไฟล์ Template PDF ของอาจารย์")
    uploaded_tpl = files.upload()
    template_path = list(uploaded_tpl.keys())[0]
    template_text_raw, _ = extract_text_and_embed(template_path)
elif USE_TEMPLATE != 0:
    raise ValueError("ค่า USE_TEMPLATE ต้องเป็น 0 หรือ 1 เท่านั้น")

raw_texts = {}

for pdf_path in pdf_paths:
    doc_id = to_ascii_id(os.path.basename(pdf_path).split('.')[0])

    # ดึงข้อความต้นฉบับของนักศึกษา
    student_text_raw, _ = extract_text_and_embed(pdf_path)

    # สลับโหมดด้วย if/elif (เขียนแบบ match-case ก็ได้ถ้าใช้ Python 3.10+)
    if USE_TEMPLATE == 0:
        # ไม่ใช้ Template
        clean_text = student_text_raw
        removed_text = ""
    elif USE_TEMPLATE == 1:
        # ใช้ Template: ลบข้อความที่เหมือน template ออก
        clean_text, removed_text = remove_template_text(
            student_text_raw,
            template_text_raw,
            return_removed=True
        )
        # แสดงข้อความที่ถูกลบ (เพื่อเช็ค/ดีบัก)
        print(f"\n--- ข้อความที่ถูกลบออกจาก Template: {doc_id} ---\n{removed_text}\n")
    else:
        raise ValueError("ค่า USE_TEMPLATE ต้องเป็น 0 หรือ 1 เท่านั้น")

    # ตัดคำ + ฝัง embedding
    tokens = word_tokenize(clean_text, engine="newmm")
    tokenized_text = " ".join(tokens)
    text_embedding = labse_model.encode(tokenized_text)

    # อัปโหลดเข้า Vector DB (ข้อความ)
    upload_text_embedding(doc_id, text_embedding)

    # เก็บข้อความสะอาดไว้ใช้เปรียบเทียบ
    raw_texts[doc_id] = clean_text

    # ฝัง image embedding และอัปโหลด
    image_embeddings = extract_images_and_embed(pdf_path)
    upload_image_embedding(doc_id, image_embeddings)


ต้องการใช้ไฟล์ Template ของอาจารย์เพื่อลบข้อความซ้ำหรือไม่?
0 = ไม่ใช้ template
1 = ใช้ template
เลือกโหมด 0 หรือ 1 : 0


In [151]:
def preprocess_text(text):
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    tokens = word_tokenize(text, engine="newmm")
    return " ".join(tokens)

def compute_lexical_similarity(text1, text2, ngram_range=(1,3)):
    t1, t2 = preprocess_text(text1), preprocess_text(text2)
    vectorizer = TfidfVectorizer(analyzer='word', ngram_range=ngram_range)
    tfidf = vectorizer.fit_transform([t1, t2])
    return cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]

def compute_hybrid_similarity(text1, text2, emb1, emb2, alpha=0.5, beta=0.5):
    semantic = cosine_similarity([emb1], [emb2])[0][0]
    lexical = compute_lexical_similarity(text1, text2)
    hybrid = alpha * semantic + beta * lexical
    return hybrid, semantic, lexical


In [159]:
def get_all_text_embeddings():
    stats = text_index.describe_index_stats()
    total_vectors = stats["total_vector_count"]
    if total_vectors == 0:
        return {}
    text_vectors = text_index.query(vector=[0]*labse_model.get_sentence_embedding_dimension(), top_k=total_vectors, include_values=True)
    return {match["id"]: np.array(match["values"]) for match in text_vectors["matches"]}

def get_all_image_embeddings():
    stats = image_index.describe_index_stats()
    total_vectors = stats["total_vector_count"]

    if total_vectors == 0:
        print("ไม่มีรูปภาพใน index")
        return {}

    #ใช้ query() เพื่อดึงค่าเวกเตอร์ทั้งหมดแทน fetch()
    image_vectors = image_index.query(
        vector=[0] * 768,
        top_k=total_vectors,
        include_values=True
    )

    return {match["id"]: np.array(match["values"]) for match in image_vectors["matches"]}




In [160]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def compare_all_documents():
    text_embeddings = get_all_text_embeddings()
    image_embeddings = get_all_image_embeddings()
    results = []

    doc_ids = list(raw_texts.keys())
    for i in range(len(doc_ids)):
        for j in range(i + 1, len(doc_ids)):
            id1, id2 = doc_ids[i], doc_ids[j]
            t1, t2 = raw_texts[id1], raw_texts[id2]
            e1, e2 = text_embeddings[f"text_{id1}"], text_embeddings[f"text_{id2}"]

            hybrid, semantic, lexical = compute_hybrid_similarity(t1, t2, e1, e2)

            # ใช้ Matching Count แทน Top-K
            image_sim = matching_count(image_embeddings, id1, id2, threshold=0.8)

            # ข้อความเกี่ยวกับการคล้ายคลึงของ Text
            if semantic < 0.7:
                text_message = "ข้อความมีความคล้ายน้อย"
            else:
                text_message = "ข้อความมีความคล้ายคลึงสูง"

            # แสดงผล Image: เปลี่ยนเป็นข้อมูลที่แสดงว่าภาพเหมือนกี่ในกี่
            keys1 = [k for k in image_embeddings if k.startswith(f"image_{id1}_")]
            keys2 = [k for k in image_embeddings if k.startswith(f"image_{id2}_")]
            total_images = len(keys1) * len(keys2)
            matched_images = int(image_sim * total_images)  # หาคู่ที่ตรงตามเกณฑ์

            # แสดงผลลัพธ์
            level = "เหมือนมาก" if hybrid >= 0.9 else \
                    "คล้ายสูง" if hybrid >= 0.75 else \
                    "คล้ายระดับกลาง" if hybrid >= 0.6 else "ไม่คล้าย"

            print("===============================================================================")
            print(f"{id1} vs {id2} [{level}]")
            print(f"   Semantic: {semantic:.4f} | Lexical: {lexical:.4f} | Hybrid: {hybrid:.4f} | Image: {image_sim:.4f}")
            print(f"   Text Message: {text_message}")
            print(f"   Image: ภาพเหมือน {matched_images} จาก {total_images} รูป")

            results.append({
                "doc_1": id1,
                "doc_2": id2,
                "hybrid_similarity": hybrid,
                "semantic_similarity": semantic,
                "lexical_similarity": lexical,
                "image_similarity": image_sim,
                "level": level,
                "text_message": text_message,
                "matched_images": matched_images,
                "total_images": total_images
            })
    return results

In [161]:
compare_results = compare_all_documents()

663380129_5 vs 663380446_4 [ไม่คล้าย]
   Semantic: 0.3071 | Lexical: 0.0105 | Hybrid: 0.1588 | Image: 0.0000
   Text Message: ข้อความมีความคล้ายน้อย
   Image: ภาพเหมือน 0 จาก 1 รูป
663380129_5 vs 663380446_5 [ไม่คล้าย]
   Semantic: 0.3660 | Lexical: 0.1309 | Hybrid: 0.2484 | Image: 1.0000
   Text Message: ข้อความมีความคล้ายน้อย
   Image: ภาพเหมือน 1 จาก 1 รูป
663380446_4 vs 663380446_5 [ไม่คล้าย]
   Semantic: 0.3161 | Lexical: 0.0000 | Hybrid: 0.1580 | Image: 1.0000
   Text Message: ข้อความมีความคล้ายน้อย
   Image: ภาพเหมือน 1 จาก 1 รูป


# Deleter


In [141]:
text_index.delete(delete_all=True)
image_index.delete(delete_all=True)

print("ลบข้อมูลใน pinecone ทั้งหมด")


ลบข้อมูลใน pinecone ทั้งหมด


In [142]:
import os
import glob

# ค้นหาไฟล์ .pdf ทั้งหมดใน /content
pdf_files = glob.glob('/content/*.pdf')

# ลบไฟล์ .pdf ทีละไฟล์
for file_path in pdf_files:
    try:
        os.remove(file_path)
        print(f'ลบไฟล์: {file_path}')
    except Exception as e:
        print(f'ไม่สามารถลบ {file_path} ได้: {e}')


ลบไฟล์: /content/663380446-5.pdf
ลบไฟล์: /content/663380129-5.pdf
ลบไฟล์: /content/663380446-4.pdf
